— 本次学习笔记为之前课程的总结，包括3个练习题。可以综合考察前期的学习效果

## 【任务一】企业收入的多样性

【题目描述】一个企业的产业收入多样性可以仿照信息熵的概念来定义收入熵指标：

 - I=−∑ip(xi)log(p(xi))其中 p(xi)是企业该年某产业收入额占该年所有产业总收入的比重。
 - 在company.csv中存有需要计算的企业和年份，在company_data.csv中存有企业、各类收入额和收入年份的信息。
 - 现请利用后一张表中的数据，在前一张表中增加一列表示该公司该年份的收入熵指标 I。

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 信息查看
df1 = pd.read_csv('company.csv')
df1.head(7)

,证券代码,日期
0,#000007,2014
1,#000403,2015
2,#000408,2016
3,#000408,2017
4,#000426,2015
5,#000426,2016
6,#000426,2017


In [3]:
print(df1.shape)
df1['证券代码'].nunique()

(1048, 2)


253

 - 共有1048行数据，公司数目为253个，说明一个公司对应好几个年份
 - 下面需要按照公司-年份进行统计

In [4]:
df2 = pd.read_csv('company_data.csv')
df2.head(10)

,证券代码,日期,收入类型,收入额
0,1,2008/12/31,1,1.084218e+10
1,1,2008/12/31,2,1.259789e+10
2,1,2008/12/31,3,1.451312e+10
3,1,2008/12/31,4,1.063843e+09
4,1,2008/12/31,5,8.513880e+08
5,1,2008/12/31,6,2.062714e+09
6,1,2008/12/31,7,-3.070010e+08
7,1,2008/12/31,8,9.605849e+09
8,1,2008/12/31,9,-8.697750e+08
9,1,2008/12/31,10,9.174519e+09


 ### - 公司信息数据需要处理：
 1. 需要将证券代码前面补0够6位后加‘#“号，和表一中的对应
 2. 收入额有负数，把直接有负数的行去除后参数统计
 3. df2中的日期取前4位，变成和df1一致

In [5]:
print('数据处理前，共有{}行'.format(df2.shape[0]))

数据处理前，共有964022行


In [6]:
df2_new = df2[df2['收入额'] > 0]
df2_new['证券代码'] = df2_new['证券代码'].apply(lambda x: '#' + str(x).zfill(6))
df2_new['日期'] = df2['日期'].str[:4].astype('int64')
print('将包含负数的行删除后，共有{}行'.format(df2_new.shape[0]))
df2_new.head(10)

将包含负数的行删除后，共有894959行


,证券代码,日期,收入类型,收入额
0,#000001,2008,1,1.084218e+10
1,#000001,2008,2,1.259789e+10
2,#000001,2008,3,1.451312e+10
3,#000001,2008,4,1.063843e+09
4,#000001,2008,5,8.513880e+08
5,#000001,2008,6,2.062714e+09
7,#000001,2008,8,9.605849e+09
9,#000001,2008,10,9.174519e+09
10,#000001,2008,11,1.080750e+10
11,#000001,2008,12,5.207130e+08


In [7]:
# 连接df1.df2_new,
res = pd.merge(df1, df2_new, on=['证券代码','日期'], how = 'left')
res.head()

,证券代码,日期,收入类型,收入额
0,#000007,2014,1.0,61461724.73
1,#000007,2014,3.0,61461724.73
2,#000007,2014,4.0,98631112.52
3,#000007,2014,5.0,80000000.00
4,#000007,2014,6.0,18631112.52


In [8]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48861 entries, 0 to 48860
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   证券代码    48861 non-null  object 
 1   日期      48861 non-null  int64  
 2   收入类型    48768 non-null  float64
 3   收入额     48768 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.9+ MB


In [9]:
# 定义一个函数计算收入熵
def incoming_entropy(incoming):
    entropy = incoming/incoming.sum()
    return -(entropy*np.log2(entropy)).sum()

In [10]:
solution = res.groupby(['证券代码','日期'])['收入额'].apply(incoming_entropy).reset_index()
solution.head(10)

,证券代码,日期,收入额
0,#000007,2014,4.429740
1,#000403,2015,4.025963
2,#000408,2016,4.066295
3,#000408,2017,-0.000000
4,#000426,2015,4.449655
5,#000426,2016,4.312071
6,#000426,2017,-0.000000
7,#000511,2012,4.506593
8,#000511,2013,4.503640
9,#000571,2014,4.124640


In [17]:
solution.rename({'收入额' : '收入熵'}, axis = 1).to_csv('company_entropy.csv', index = False)

In [18]:
# 读取，查看是否写入成功
company = pd.read_csv('company_entropy.csv')
company.head(3)

,证券代码,日期,收入熵
0,#000007,2014,4.429740
1,#000403,2015,4.025963
2,#000408,2016,4.066295


## 总结：
1. 重点考察了merge和groupby的知识点
2. 另外需要对数据进行预处理，好在都比较简单

## 【任务二】组队学习信息表的变换

【题目描述】请把组队学习的队伍信息表变换为如下形态，其中“是否队长”一列取1表示队长，否则为0
	是否队长	队伍名称	    昵称    	编号
0	1	    你说的都对队	山枫叶纷飞	5
1	0	    你说的都对队	蔡	        6
2	0	    你说的都对队	安慕希	    7
3	0	    你说的都对队	信仰	    8
4	0	    你说的都对队	biubiu🙈🙈	20
...	...	    ...	        ...	        ...
141	0	    七星联盟	    Daisy	    63
142	0	    七星联盟    	One Better	131
143	0	    七星联盟    	rain	    112
144	1	    应如是	    思无邪	    54
145	0	    应如是	    Justzer0	58

In [35]:
student = pd.read_excel('组队信息汇总表（Pandas).xlsx')
student.head()

,所在群,队伍名称,队长编号,队长_群昵称,队员1 编号,队员_群昵称,队员2 编号,队员_群昵称.1,队员3 编号,队员_群昵称.2,...,队员6 编号,队员_群昵称.5,队员7 编号,队员_群昵称.6,队员8 编号,队员_群昵称.7,队员9 编号,队员_群昵称.8,队员10编号,队员_群昵称.9
0,Pandas数据分析,你说的都对队,5,山枫叶纷飞,6,蔡,7.0,安慕希,8.0,信仰,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pandas数据分析,熊猫人,175,鱼呲呲,44,Heaven,37.0,吕青,50.0,余柳成荫,...,25.0,Never say never,55.0,K,120.0,Y.,28.0,X.Y.Q,151.0,swrong
2,Pandas数据分析,中国移不动,107,Y's,124,🥕,75.0,Vito,146.0,张小五,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pandas数据分析,panda,11,太下真君,35,柚子,108.0,My,42.0,星星点灯,...,157.0,Zys,158.0,不器,102.0,嘉平佑染,NaN,NaN,NaN,NaN
4,Pandas数据分析,一路向北,13,黄元帅,15,化,16.0,未期,18.0,太陽光下,...,23.0,🚀,169.0,听风,189.0,Cappuccino,NaN,NaN,NaN,NaN


In [36]:
# columns查看
student.columns

Index(['所在群', '队伍名称', '队长编号', '队长_群昵称', '队员1 编号', '队员_群昵称', '队员2 编号',
       '队员_群昵称.1', '队员3 编号', '队员_群昵称.2', '队员4 编号', '队员_群昵称.3', '队员5 编号',
       '队员_群昵称.4', '队员6 编号', '队员_群昵称.5', '队员7 编号', '队员_群昵称.6', '队员8 编号',
       '队员_群昵称.7', '队员9 编号', '队员_群昵称.8', '队员10编号', '队员_群昵称.9'],
      dtype='object')

 ### step1. 数据预处理
 - 字段所在群可以删掉
 - 重命名字段，队长和队员使用chief和staff区分

In [37]:
student.drop(columns = ['所在群', ], inplace = True)
student.head(3)

,队伍名称,队长编号,队长_群昵称,队员1 编号,队员_群昵称,队员2 编号,队员_群昵称.1,队员3 编号,队员_群昵称.2,队员4 编号,...,队员6 编号,队员_群昵称.5,队员7 编号,队员_群昵称.6,队员8 编号,队员_群昵称.7,队员9 编号,队员_群昵称.8,队员10编号,队员_群昵称.9
0,你说的都对队,5,山枫叶纷飞,6,蔡,7.0,安慕希,8.0,信仰,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,熊猫人,175,鱼呲呲,44,Heaven,37.0,吕青,50.0,余柳成荫,82.0,...,25.0,Never say never,55.0,K,120.0,Y.,28.0,X.Y.Q,151.0,swrong
2,中国移不动,107,Y's,124,🥕,75.0,Vito,146.0,张小五,186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
chief = np.array(['队伍名称','编号_chief','昵称_chief'])
staff = np.array([[f'编号_staff{i}', f'昵称_staff{i}']for i in range(1,11)]).flatten()
student.columns = np.r_[chief, staff]
student.head()

,队伍名称,编号_chief,昵称_chief,编号_staff1,昵称_staff1,编号_staff2,昵称_staff2,编号_staff3,昵称_staff3,编号_staff4,...,编号_staff6,昵称_staff6,编号_staff7,昵称_staff7,编号_staff8,昵称_staff8,编号_staff9,昵称_staff9,编号_staff10,昵称_staff10
0,你说的都对队,5,山枫叶纷飞,6,蔡,7.0,安慕希,8.0,信仰,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,熊猫人,175,鱼呲呲,44,Heaven,37.0,吕青,50.0,余柳成荫,82.0,...,25.0,Never say never,55.0,K,120.0,Y.,28.0,X.Y.Q,151.0,swrong
2,中国移不动,107,Y's,124,🥕,75.0,Vito,146.0,张小五,186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,panda,11,太下真君,35,柚子,108.0,My,42.0,星星点灯,45.0,...,157.0,Zys,158.0,不器,102.0,嘉平佑染,NaN,NaN,NaN,NaN
4,一路向北,13,黄元帅,15,化,16.0,未期,18.0,太陽光下,19.0,...,23.0,🚀,169.0,听风,189.0,Cappuccino,NaN,NaN,NaN,NaN


 ### step2. 使用wide_to_long将长表转宽表

In [48]:
solution = pd.wide_to_long(student.reset_index(),
                          stubnames = ['昵称','编号'],
                          i = ['index','队伍名称'],
                          j = '是否队长',
                           sep = '_',
                          suffix = '.+').dropna().reset_index().drop(columns='index' )
solution.head(8)

,队伍名称,是否队长,昵称,编号
0,你说的都对队,chief,山枫叶纷飞,5.0
1,你说的都对队,staff1,蔡,6.0
2,你说的都对队,staff2,安慕希,7.0
3,你说的都对队,staff3,信仰,8.0
4,你说的都对队,staff4,biubiu🙈🙈,20.0
5,熊猫人,chief,鱼呲呲,175.0
6,熊猫人,staff1,Heaven,44.0
7,熊猫人,staff2,吕青,37.0


In [49]:
solution['是否队长'] = solution['是否队长'].apply(lambda x: 1 if x == 'chief' else 0)
solution.head(10)

,队伍名称,是否队长,昵称,编号
0,你说的都对队,1,山枫叶纷飞,5.0
1,你说的都对队,0,蔡,6.0
2,你说的都对队,0,安慕希,7.0
3,你说的都对队,0,信仰,8.0
4,你说的都对队,0,biubiu🙈🙈,20.0
5,熊猫人,1,鱼呲呲,175.0
6,熊猫人,0,Heaven,44.0
7,熊猫人,0,吕青,37.0
8,熊猫人,0,余柳成荫,50.0
9,熊猫人,0,Kuroe,82.0


In [50]:
solution['编号'] = solution['编号'].astype('int8')
solution.reindex(columns=['是否队长','队伍名称','昵称','编号'])
solution.head()

,队伍名称,是否队长,昵称,编号
0,你说的都对队,1,山枫叶纷飞,5
1,你说的都对队,0,蔡,6
2,你说的都对队,0,安慕希,7
3,你说的都对队,0,信仰,8
4,你说的都对队,0,biubiu🙈🙈,20


## 【任务三】美国大选投票情况

【题目描述】两张数据表中分别给出了美国各县（county）的人口数以及大选的投票情况，请解决以下问题：

有多少县满足总投票数超过县人口数的一半
把州（state）作为行索引，把投票候选人作为列名，列名的顺序按照候选人在全美的总票数由高到低排序，行列对应的元素为该候选人在该州获得的总票数